In [1]:
#from opentrons import protocol_api
import opentrons.execute
import time
import json

protocol = opentrons.execute.get_protocol_api('2.13')
#尽管我在使用2.14 的机器，2.13的api也是可以的。2.14版本之后似乎禁用了直接使用jupyter 调用
protocol.home()

/data/robot_settings.json not found. Loading defaults
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
/data/deck_calibration.json not found. Loading defaults


In [2]:
with open('./xinglab_pcr96well_semiskirt_280ul.json') as labware_file:
    labware_def = json.load(labware_file)
    well_plate = protocol.load_labware_from_definition(labware_def,location='2')

with open('./xinglab_axygen_96_diytiprack_10ul.json') as labware_file:
    labware_def = json.load(labware_file)
    tipracks = protocol.load_labware_from_definition(labware_def,location=['1','3'])

pipette = protocol.load_instrument('p20_multi_gen2','right',tip_racks=[tipracks])

In [ ]:
pipette.flow_rate.aspirate = 5  
pipette.flow_rate.dispense = 5
columns = ['3','4','5','6']

volumes1 = [8]

pipette.pick_up_tip()

for col in columns:
    for i, volume in enumerate(volumes1):
        pipette.aspirate(volume, well_plate.columns_by_name()['1'][0].bottom()) 
        pipette.dispense(volume, well_plate.columns_by_name()[col][0].bottom())
        pipette.move_to(well_plate.columns_by_name()[col][0].bottom(20))
        pipette.blow_out()
        
pipette.drop_tip()
        
volumes2 = [2]
for col in columns:
    for i, volume in enumerate(volumes2):
        pipette.pick_up_tip()
        pipette.aspirate(volume, well_plate.columns_by_name()['12'][0].bottom(-1)) 
        pipette.dispense(volume, well_plate.columns_by_name()[col][0].bottom())
        pipette.mix(10, 8, rate=10)
        pipette.move_to(well_plate.columns_by_name()[col][0].bottom(30))
        pipette.blow_out()
        
        pipette.move_to(well_plate.columns_by_name()[col][0].top(20))
        time.sleep(3)
        
        pipette.drop_tip()